In [1]:
import pandas as pd
import numpy as np
import random as rand
import time
from collections import Counter

This notebook will attempt to check the performance of
Agg_kjm([[B(k,i), C(k), D(k)],[D(j,l,m), C(j)]],R(i,l)) versus 
Agg_kj([[B(k,i), C(k), D(k)], Agg_m([[D(j=b, l,m)]], T(j, l)), [T(j, l), C(j)]], R(i,l)) versus
Agg_kj([Agg_m([[D(k,j,l,m)]],T(k,j,l)), [B(k,i), C(k), T(k)], [T(j, l), C(j)]], R(i,l))

In [2]:
domainSize = 50
B_Size = 10
C_Size = 1000**2
D_Size = domainSize**4
B, C, D = dict(), dict(), dict()
for _ in range(B_Size):
    column = rand.randint(0,domainSize)
    row = rand.randint(0,domainSize)
    if column not in B:
        B[column] = set()
    B[column].add(row)
    
    
for _ in range(C_Size):
    column = rand.randint(0,domainSize)
    row = rand.randint(0,domainSize)
    if column not in C:
        C[column] = set()
    C[column].add(row)
    
for _ in range(D_Size):
    column = rand.randint(0,domainSize)
    row = tuple([rand.randint(0,domainSize) for _ in range(3)])
    if column not in D:
        D[column] = set()
    D[column].add(row)

In [3]:
%%timeit
aggResults1 = dict()
for k in B.keys():
    if (k not in C) or (k not in D):
        continue
    for i in B[k]:
        for j,l,m in D[k]:
            x = [1 for _ in range(100)] # assuming some lookup cost associated with C[k]
            if j in C[k]:
                if (i,l) not in aggResults1:
                    aggResults1[(i,l)] = 1
                else:
                    aggResults1[(i,l)] += 1

2.19 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
aggResults2 = dict()
for k in B.keys():
    if (k not in C) or (k not in D):
        continue
    for i in B[k]:
        t = dict()
        for j,l,m in D[k]:
            if (j,l) not in t:
                t[(j,l)] = 1
            else:
                t[(j,l)] += 1
        for j,l in t.keys():
            x = [1 for _ in range(100)] # assuming some lookup cost associated with C[k]
            if j in C[k]:
                if (i,l) not in aggResults2:
                    aggResults2[(i,l)] = t[(j,l)]
                else:
                    aggResults2[(i,l)] += t[(j,l)]

293 ms ± 6.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
aggResults3 = dict()
t = dict()
for k, v in D.items():
    if k not in t:
        t[k] = dict()
    for j,l,m in v:        
        if (j,l) not in t[k]:
            t[k][(j,l)] = 1
        else:
            t[k][(j,l)] += 1
for k in B.keys():
    if (k not in C) or (k not in t):
        continue
    for i in B[k]:
        for j,l in t[k].keys():
            x = [1 for _ in range(100)] # assuming some lookup cost associated with C[k]
            if j in C[k]:
                if (i,l) not in aggResults3:
                    aggResults3[(i,l)] = t[k][(j,l)]
                else:
                    aggResults3[(i,l)] += t[k][(j,l)]

1.4 s ± 33.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
for k in aggResults1.keys():
    if aggResults1[k] != aggResults2[k]:
        print("PROBLEM")
    if aggResults1[k] != aggResults3[k]:
        print("PROBLEM")

NameError: name 'aggResults1' is not defined